# Catboost

CatBoost stands for Categorical Boosting. It takes a weak base classifier (a small decision tree) and boosts the weak classifier by learning it many times, each time trying to correct the previous classifier's errors. They are then combined to a boosted classifier. Because the new trees are trained on the difference between true labels and the previous prediction, this method is called gradient boosting. The learning rate will determine the speed of boosting: A small learning rate means that one needs many trees to approximate the training data, but will usually prevent overfitting.

What makes CatBoost a particularly interesting classifier is its handling of categorical values: There is no prior transformation of categorical values necessary (e.g. one-hot-encoding), but CatBoost takes care of all of this by itself. Moreover, CatBoost can combine 2 categorical features to a new one, if this helps the learning process (if e.g. there are strong correlations between two features). It can even do this with continuous features by splitting them into intervals and treating the intervals as categorical information.

Tree-based methods can give a straightforward interpretation of the variable importance because the earlier a variable appears in the split, the more important it is. CatBoost is no exception to this and therefore has its own internal variable importance as an addition to "external" importance estimators like e.g. SHAP.

 <a href="learningsys.org/nips17/assets/papers/paper_11.pdf">Publication on CatBoost</a>



## 1 Installing Catboost and Preparing the Data
We will test CatBoost on the famous Titanic data set. The target variable of the Titanic data is the "Survived" column and the other columns include categorical (e.g. passenger class Pclass and Sex) and numeric (e.g. Age) features. Some of the categorical features are strings, while others are integers. But CatBoost does not bother with that and transforms the categorical features on its own, meaning that we do not have to prepare the categorical features. The test_df has no true labels (y_test) because the data is from a kaggle competition, i.e. you have to estimate the labels yourself.

In [1]:
import sys
!{sys.executable} -m pip install catboost
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import catboost
from catboost.datasets import titanic
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
import numpy as np

train_df, test_df = titanic()
print(type(train_df), train_df.shape, test_df.shape)
train_df.head()

<class 'pandas.core.frame.DataFrame'> (891, 12) (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Search for missing values
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

As recommended in the official CatBoost tutorial, missing values should be filled with a value way outside of their distribution. Hence, the algorithm will recognise that this is markedly different from the other values.

In [4]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

In [5]:
## Train Test Split

X = train_df.drop('Survived', axis=1)
y = train_df.Survived

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)
X_test = test_df

# One has to tell CatBoost which features are categorical because it could also treat continuous ones like categorical if one wishes to do so
categorical_features_indices = np.where(X.dtypes != float)[0]

In [6]:
print(y_train.mean(), y_validation.mean())

0.3787425149700599 0.3991031390134529


Because the data is split fairly even in a 40:60 ratio, simply "predicting" that everyone survived or everyone drowned will not result in a good classifier. Hence, using accuracy as a metric can be justified.

## 2 Training the Model

In [7]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [8]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    logging_level='Verbose',  # you can uncomment this for text output, this prints status of training process
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.028683
0:	learn: 0.6739988	test: 0.6742630	best: 0.6742630 (0)	total: 62.9ms	remaining: 1m 2s
1:	learn: 0.6589013	test: 0.6592240	best: 0.6592240 (1)	total: 65.6ms	remaining: 32.7s
2:	learn: 0.6421502	test: 0.6426778	best: 0.6426778 (2)	total: 69.9ms	remaining: 23.2s
3:	learn: 0.6297276	test: 0.6302310	best: 0.6302310 (3)	total: 73.1ms	remaining: 18.2s
4:	learn: 0.6147184	test: 0.6198228	best: 0.6198228 (4)	total: 77.3ms	remaining: 15.4s
5:	learn: 0.6017730	test: 0.6073627	best: 0.6073627 (5)	total: 79.4ms	remaining: 13.2s
6:	learn: 0.5885309	test: 0.5956000	best: 0.5956000 (6)	total: 81.4ms	remaining: 11.6s
7:	learn: 0.5783200	test: 0.5858523	best: 0.5858523 (7)	total: 83.7ms	remaining: 10.4s
8:	learn: 0.5665895	test: 0.5743842	best: 0.5743842 (8)	total: 87.3ms	remaining: 9.61s
9:	learn: 0.5575381	test: 0.5662283	best: 0.5662283 (9)	total: 89.6ms	remaining: 8.87s
10:	learn: 0.5491045	test: 0.5575176	best: 0.5575176 (10)	total: 92.2ms	remaining: 8.29s
11:	learn: 

111:	learn: 0.3646549	test: 0.4087192	best: 0.4087192 (111)	total: 350ms	remaining: 2.77s
112:	learn: 0.3634674	test: 0.4075590	best: 0.4075590 (112)	total: 352ms	remaining: 2.76s
113:	learn: 0.3630065	test: 0.4073434	best: 0.4073434 (113)	total: 355ms	remaining: 2.76s
114:	learn: 0.3624458	test: 0.4074634	best: 0.4073434 (113)	total: 357ms	remaining: 2.75s
115:	learn: 0.3616972	test: 0.4071796	best: 0.4071796 (115)	total: 360ms	remaining: 2.74s
116:	learn: 0.3615271	test: 0.4071387	best: 0.4071387 (116)	total: 362ms	remaining: 2.73s
117:	learn: 0.3614277	test: 0.4072280	best: 0.4071387 (116)	total: 364ms	remaining: 2.72s
118:	learn: 0.3603144	test: 0.4062959	best: 0.4062959 (118)	total: 366ms	remaining: 2.71s
119:	learn: 0.3599630	test: 0.4062330	best: 0.4062330 (119)	total: 368ms	remaining: 2.7s
120:	learn: 0.3592807	test: 0.4058909	best: 0.4058909 (120)	total: 371ms	remaining: 2.69s
121:	learn: 0.3587751	test: 0.4056309	best: 0.4056309 (121)	total: 375ms	remaining: 2.7s
122:	learn: 

245:	learn: 0.3120558	test: 0.3947056	best: 0.3945455 (239)	total: 677ms	remaining: 2.08s
246:	learn: 0.3118030	test: 0.3945753	best: 0.3945455 (239)	total: 680ms	remaining: 2.07s
247:	learn: 0.3115541	test: 0.3945034	best: 0.3945034 (247)	total: 682ms	remaining: 2.07s
248:	learn: 0.3113836	test: 0.3945553	best: 0.3945034 (247)	total: 684ms	remaining: 2.06s
249:	learn: 0.3109114	test: 0.3945776	best: 0.3945034 (247)	total: 687ms	remaining: 2.06s
250:	learn: 0.3105904	test: 0.3946490	best: 0.3945034 (247)	total: 689ms	remaining: 2.06s
251:	learn: 0.3104614	test: 0.3946911	best: 0.3945034 (247)	total: 692ms	remaining: 2.05s
252:	learn: 0.3102894	test: 0.3946865	best: 0.3945034 (247)	total: 694ms	remaining: 2.05s
253:	learn: 0.3098849	test: 0.3944783	best: 0.3944783 (253)	total: 696ms	remaining: 2.04s
254:	learn: 0.3096090	test: 0.3945152	best: 0.3944783 (253)	total: 698ms	remaining: 2.04s
255:	learn: 0.3095389	test: 0.3945467	best: 0.3944783 (253)	total: 701ms	remaining: 2.04s
256:	learn

344:	learn: 0.2816979	test: 0.3927250	best: 0.3925689 (343)	total: 930ms	remaining: 1.77s
345:	learn: 0.2812803	test: 0.3932255	best: 0.3925689 (343)	total: 934ms	remaining: 1.76s
346:	learn: 0.2803450	test: 0.3933986	best: 0.3925689 (343)	total: 937ms	remaining: 1.76s
347:	learn: 0.2796085	test: 0.3934529	best: 0.3925689 (343)	total: 947ms	remaining: 1.77s
348:	learn: 0.2793429	test: 0.3935490	best: 0.3925689 (343)	total: 950ms	remaining: 1.77s
349:	learn: 0.2786006	test: 0.3937183	best: 0.3925689 (343)	total: 960ms	remaining: 1.78s
350:	learn: 0.2777571	test: 0.3940834	best: 0.3925689 (343)	total: 963ms	remaining: 1.78s
351:	learn: 0.2776090	test: 0.3940769	best: 0.3925689 (343)	total: 968ms	remaining: 1.78s
352:	learn: 0.2767992	test: 0.3934999	best: 0.3925689 (343)	total: 974ms	remaining: 1.78s
353:	learn: 0.2764869	test: 0.3933342	best: 0.3925689 (343)	total: 977ms	remaining: 1.78s
354:	learn: 0.2757839	test: 0.3935131	best: 0.3925689 (343)	total: 980ms	remaining: 1.78s
355:	learn

...

556:	learn: 0.2156300	test: 0.4035766	best: 0.3925689 (343)	total: 1.66s	remaining: 1.32s
557:	learn: 0.2154524	test: 0.4032750	best: 0.3925689 (343)	total: 1.66s	remaining: 1.32s
558:	learn: 0.2153958	test: 0.4033683	best: 0.3925689 (343)	total: 1.67s	remaining: 1.31s
559:	learn: 0.2153603	test: 0.4033591	best: 0.3925689 (343)	total: 1.67s	remaining: 1.31s
560:	learn: 0.2153034	test: 0.4033728	best: 0.3925689 (343)	total: 1.67s	remaining: 1.31s
561:	learn: 0.2152633	test: 0.4032866	best: 0.3925689 (343)	total: 1.67s	remaining: 1.3s
562:	learn: 0.2148556	test: 0.4036073	best: 0.3925689 (343)	total: 1.68s	remaining: 1.3s
563:	learn: 0.2148436	test: 0.4035961	best: 0.3925689 (343)	total: 1.68s	remaining: 1.3s
564:	learn: 0.2148228	test: 0.4036479	best: 0.3925689 (343)	total: 1.68s	remaining: 1.3s
565:	learn: 0.2147965	test: 0.4035684	best: 0.3925689 (343)	total: 1.69s	remaining: 1.29s
566:	learn: 0.2146438	test: 0.4035601	best: 0.3925689 (343)	total: 1.69s	remaining: 1.29s
567:	learn: 0.

657:	learn: 0.1962348	test: 0.4091966	best: 0.3925689 (343)	total: 1.92s	remaining: 998ms
658:	learn: 0.1961597	test: 0.4091615	best: 0.3925689 (343)	total: 1.92s	remaining: 995ms
659:	learn: 0.1956394	test: 0.4092077	best: 0.3925689 (343)	total: 1.93s	remaining: 992ms
660:	learn: 0.1956237	test: 0.4092356	best: 0.3925689 (343)	total: 1.94s	remaining: 996ms
661:	learn: 0.1954545	test: 0.4093616	best: 0.3925689 (343)	total: 1.96s	remaining: 999ms
662:	learn: 0.1951190	test: 0.4097803	best: 0.3925689 (343)	total: 1.96s	remaining: 996ms
663:	learn: 0.1950099	test: 0.4097903	best: 0.3925689 (343)	total: 1.97s	remaining: 996ms
664:	learn: 0.1948989	test: 0.4097491	best: 0.3925689 (343)	total: 1.97s	remaining: 992ms
665:	learn: 0.1945474	test: 0.4100763	best: 0.3925689 (343)	total: 1.97s	remaining: 989ms
666:	learn: 0.1944861	test: 0.4100757	best: 0.3925689 (343)	total: 1.98s	remaining: 986ms
667:	learn: 0.1943174	test: 0.4101521	best: 0.3925689 (343)	total: 1.98s	remaining: 983ms
668:	learn

765:	learn: 0.1753264	test: 0.4165993	best: 0.3925689 (343)	total: 2.26s	remaining: 690ms
766:	learn: 0.1752084	test: 0.4166235	best: 0.3925689 (343)	total: 2.26s	remaining: 687ms
767:	learn: 0.1749531	test: 0.4170032	best: 0.3925689 (343)	total: 2.26s	remaining: 684ms
768:	learn: 0.1749129	test: 0.4170263	best: 0.3925689 (343)	total: 2.27s	remaining: 681ms
769:	learn: 0.1745286	test: 0.4176999	best: 0.3925689 (343)	total: 2.27s	remaining: 678ms
770:	learn: 0.1745228	test: 0.4177098	best: 0.3925689 (343)	total: 2.27s	remaining: 675ms
771:	learn: 0.1741362	test: 0.4176875	best: 0.3925689 (343)	total: 2.28s	remaining: 673ms
772:	learn: 0.1737494	test: 0.4178495	best: 0.3925689 (343)	total: 2.28s	remaining: 670ms
773:	learn: 0.1737326	test: 0.4179174	best: 0.3925689 (343)	total: 2.28s	remaining: 667ms
774:	learn: 0.1735426	test: 0.4178826	best: 0.3925689 (343)	total: 2.29s	remaining: 663ms
775:	learn: 0.1732704	test: 0.4181271	best: 0.3925689 (343)	total: 2.29s	remaining: 660ms
776:	learn

868:	learn: 0.1584100	test: 0.4254165	best: 0.3925689 (343)	total: 2.54s	remaining: 383ms
869:	learn: 0.1582611	test: 0.4252118	best: 0.3925689 (343)	total: 2.54s	remaining: 380ms
870:	learn: 0.1580364	test: 0.4255225	best: 0.3925689 (343)	total: 2.54s	remaining: 377ms
871:	learn: 0.1577863	test: 0.4264693	best: 0.3925689 (343)	total: 2.55s	remaining: 374ms
872:	learn: 0.1575010	test: 0.4266996	best: 0.3925689 (343)	total: 2.55s	remaining: 371ms
873:	learn: 0.1572197	test: 0.4268858	best: 0.3925689 (343)	total: 2.55s	remaining: 368ms
874:	learn: 0.1571717	test: 0.4268197	best: 0.3925689 (343)	total: 2.55s	remaining: 365ms
875:	learn: 0.1570976	test: 0.4269053	best: 0.3925689 (343)	total: 2.56s	remaining: 362ms
876:	learn: 0.1569863	test: 0.4270429	best: 0.3925689 (343)	total: 2.56s	remaining: 359ms
877:	learn: 0.1566991	test: 0.4269143	best: 0.3925689 (343)	total: 2.56s	remaining: 356ms
878:	learn: 0.1566554	test: 0.4270517	best: 0.3925689 (343)	total: 2.56s	remaining: 353ms
879:	learn

980:	learn: 0.1404468	test: 0.4366938	best: 0.3925689 (343)	total: 2.84s	remaining: 55.1ms
981:	learn: 0.1403263	test: 0.4366148	best: 0.3925689 (343)	total: 2.85s	remaining: 52.2ms
982:	learn: 0.1402955	test: 0.4365037	best: 0.3925689 (343)	total: 2.85s	remaining: 49.3ms
983:	learn: 0.1401256	test: 0.4366547	best: 0.3925689 (343)	total: 2.85s	remaining: 46.4ms
984:	learn: 0.1400961	test: 0.4366136	best: 0.3925689 (343)	total: 2.86s	remaining: 43.5ms
985:	learn: 0.1398101	test: 0.4368906	best: 0.3925689 (343)	total: 2.86s	remaining: 40.6ms
986:	learn: 0.1396404	test: 0.4370100	best: 0.3925689 (343)	total: 2.86s	remaining: 37.7ms
987:	learn: 0.1393904	test: 0.4370537	best: 0.3925689 (343)	total: 2.87s	remaining: 34.8ms
988:	learn: 0.1392551	test: 0.4371019	best: 0.3925689 (343)	total: 2.87s	remaining: 31.9ms
989:	learn: 0.1390832	test: 0.4368715	best: 0.3925689 (343)	total: 2.87s	remaining: 29ms
990:	learn: 0.1390399	test: 0.4367513	best: 0.3925689 (343)	total: 2.87s	remaining: 26.1ms
9

One can see that after 344 iterations, further boosting leads to a worse generalisation error, i.e. to overfitting. Hence, CatBoost chooses the 344th iteration as the best model. We use that model as a starting point for cross validation. Note that CatBoosts cross validation wants a catboost.core.Pool object as input that combines the features X, the target y and the categorical index.

In [9]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    fold_count = 4, # number of cross validations
    logging_level="Verbose" # tells us about the intermediary results
)

Training on fold [0/4]
0:	learn: 0.6761669	test: 0.6768083	best: 0.6768083 (0)	total: 18.6ms	remaining: 18.6s
1:	learn: 0.6583334	test: 0.6608511	best: 0.6608511 (1)	total: 27.1ms	remaining: 13.5s
2:	learn: 0.6423837	test: 0.6461866	best: 0.6461866 (2)	total: 34.5ms	remaining: 11.5s
3:	learn: 0.6283679	test: 0.6329292	best: 0.6329292 (3)	total: 39ms	remaining: 9.72s
4:	learn: 0.6144513	test: 0.6195135	best: 0.6195135 (4)	total: 43.6ms	remaining: 8.67s
5:	learn: 0.6026594	test: 0.6083604	best: 0.6083604 (5)	total: 48.9ms	remaining: 8.1s
6:	learn: 0.5919314	test: 0.5974993	best: 0.5974993 (6)	total: 51.6ms	remaining: 7.32s
7:	learn: 0.5802086	test: 0.5875367	best: 0.5875367 (7)	total: 56.7ms	remaining: 7.03s
8:	learn: 0.5671302	test: 0.5774444	best: 0.5774444 (8)	total: 61.5ms	remaining: 6.77s
9:	learn: 0.5613196	test: 0.5718529	best: 0.5718529 (9)	total: 63ms	remaining: 6.24s
10:	learn: 0.5524883	test: 0.5634700	best: 0.5634700 (10)	total: 70.9ms	remaining: 6.37s
11:	learn: 0.5420783	te

97:	learn: 0.3561392	test: 0.4216589	best: 0.4214465 (95)	total: 524ms	remaining: 4.82s
98:	learn: 0.3552380	test: 0.4212137	best: 0.4212137 (98)	total: 529ms	remaining: 4.82s
99:	learn: 0.3547494	test: 0.4210964	best: 0.4210964 (99)	total: 533ms	remaining: 4.8s
100:	learn: 0.3544346	test: 0.4208324	best: 0.4208324 (100)	total: 536ms	remaining: 4.77s
101:	learn: 0.3537253	test: 0.4208944	best: 0.4208324 (100)	total: 543ms	remaining: 4.78s
102:	learn: 0.3529604	test: 0.4208832	best: 0.4208324 (100)	total: 548ms	remaining: 4.77s
103:	learn: 0.3519260	test: 0.4206761	best: 0.4206761 (103)	total: 557ms	remaining: 4.8s
104:	learn: 0.3513192	test: 0.4205804	best: 0.4205804 (104)	total: 562ms	remaining: 4.79s
105:	learn: 0.3506273	test: 0.4207990	best: 0.4205804 (104)	total: 571ms	remaining: 4.81s
106:	learn: 0.3498566	test: 0.4208351	best: 0.4205804 (104)	total: 575ms	remaining: 4.8s
107:	learn: 0.3492378	test: 0.4206773	best: 0.4205804 (104)	total: 579ms	remaining: 4.78s
108:	learn: 0.34839

198:	learn: 0.3099827	test: 0.4155047	best: 0.4151780 (185)	total: 1.07s	remaining: 4.31s
199:	learn: 0.3099378	test: 0.4155051	best: 0.4151780 (185)	total: 1.07s	remaining: 4.3s
200:	learn: 0.3096826	test: 0.4156289	best: 0.4151780 (185)	total: 1.08s	remaining: 4.31s
201:	learn: 0.3094587	test: 0.4155428	best: 0.4151780 (185)	total: 1.09s	remaining: 4.32s
202:	learn: 0.3091965	test: 0.4155100	best: 0.4151780 (185)	total: 1.1s	remaining: 4.31s
203:	learn: 0.3089981	test: 0.4151900	best: 0.4151780 (185)	total: 1.1s	remaining: 4.31s
204:	learn: 0.3085965	test: 0.4150236	best: 0.4150236 (204)	total: 1.11s	remaining: 4.29s
205:	learn: 0.3082000	test: 0.4155330	best: 0.4150236 (204)	total: 1.11s	remaining: 4.28s
206:	learn: 0.3081189	test: 0.4154364	best: 0.4150236 (204)	total: 1.12s	remaining: 4.29s
207:	learn: 0.3076681	test: 0.4154269	best: 0.4150236 (204)	total: 1.12s	remaining: 4.28s
208:	learn: 0.3076370	test: 0.4154412	best: 0.4150236 (204)	total: 1.13s	remaining: 4.26s
209:	learn: 0

294:	learn: 0.2773919	test: 0.4167648	best: 0.4150236 (204)	total: 1.63s	remaining: 3.9s
295:	learn: 0.2773428	test: 0.4167625	best: 0.4150236 (204)	total: 1.64s	remaining: 3.9s
296:	learn: 0.2770425	test: 0.4166856	best: 0.4150236 (204)	total: 1.64s	remaining: 3.89s
297:	learn: 0.2766801	test: 0.4167605	best: 0.4150236 (204)	total: 1.65s	remaining: 3.88s
298:	learn: 0.2763049	test: 0.4169842	best: 0.4150236 (204)	total: 1.65s	remaining: 3.88s
299:	learn: 0.2759852	test: 0.4173463	best: 0.4150236 (204)	total: 1.66s	remaining: 3.87s
300:	learn: 0.2756637	test: 0.4174560	best: 0.4150236 (204)	total: 1.66s	remaining: 3.86s
301:	learn: 0.2751539	test: 0.4174426	best: 0.4150236 (204)	total: 1.67s	remaining: 3.85s
302:	learn: 0.2748412	test: 0.4173882	best: 0.4150236 (204)	total: 1.67s	remaining: 3.85s
303:	learn: 0.2746520	test: 0.4174950	best: 0.4150236 (204)	total: 1.68s	remaining: 3.84s
304:	learn: 0.2741592	test: 0.4173315	best: 0.4150236 (204)	total: 1.68s	remaining: 3.83s
305:	learn: 

389:	learn: 0.2465044	test: 0.4192620	best: 0.4150236 (204)	total: 2.2s	remaining: 3.44s
390:	learn: 0.2461573	test: 0.4194118	best: 0.4150236 (204)	total: 2.21s	remaining: 3.44s
391:	learn: 0.2458863	test: 0.4196550	best: 0.4150236 (204)	total: 2.21s	remaining: 3.43s
392:	learn: 0.2458480	test: 0.4197132	best: 0.4150236 (204)	total: 2.22s	remaining: 3.42s
393:	learn: 0.2455651	test: 0.4197800	best: 0.4150236 (204)	total: 2.22s	remaining: 3.42s
394:	learn: 0.2449520	test: 0.4196968	best: 0.4150236 (204)	total: 2.23s	remaining: 3.41s
395:	learn: 0.2442865	test: 0.4193730	best: 0.4150236 (204)	total: 2.23s	remaining: 3.4s
396:	learn: 0.2441380	test: 0.4195962	best: 0.4150236 (204)	total: 2.24s	remaining: 3.4s
397:	learn: 0.2439957	test: 0.4198873	best: 0.4150236 (204)	total: 2.24s	remaining: 3.39s
398:	learn: 0.2438195	test: 0.4198992	best: 0.4150236 (204)	total: 2.25s	remaining: 3.38s
399:	learn: 0.2434223	test: 0.4200363	best: 0.4150236 (204)	total: 2.26s	remaining: 3.38s
400:	learn: 0

...

575:	learn: 0.2062496	test: 0.4532716	best: 0.4329959 (200)	total: 3.79s	remaining: 2.79s
576:	learn: 0.2062151	test: 0.4533438	best: 0.4329959 (200)	total: 3.8s	remaining: 2.79s
577:	learn: 0.2058088	test: 0.4534146	best: 0.4329959 (200)	total: 3.81s	remaining: 2.78s
578:	learn: 0.2055292	test: 0.4534992	best: 0.4329959 (200)	total: 3.81s	remaining: 2.77s
579:	learn: 0.2053731	test: 0.4534799	best: 0.4329959 (200)	total: 3.82s	remaining: 2.77s
580:	learn: 0.2053213	test: 0.4534697	best: 0.4329959 (200)	total: 3.82s	remaining: 2.76s
581:	learn: 0.2051173	test: 0.4536006	best: 0.4329959 (200)	total: 3.83s	remaining: 2.75s
582:	learn: 0.2048930	test: 0.4537239	best: 0.4329959 (200)	total: 3.84s	remaining: 2.74s
583:	learn: 0.2047132	test: 0.4536814	best: 0.4329959 (200)	total: 3.84s	remaining: 2.74s
584:	learn: 0.2043622	test: 0.4538858	best: 0.4329959 (200)	total: 3.85s	remaining: 2.73s
585:	learn: 0.2034866	test: 0.4535530	best: 0.4329959 (200)	total: 3.86s	remaining: 2.73s
586:	learn:

672:	learn: 0.1851233	test: 0.4570304	best: 0.4329959 (200)	total: 4.4s	remaining: 2.14s
673:	learn: 0.1847874	test: 0.4572075	best: 0.4329959 (200)	total: 4.41s	remaining: 2.13s
674:	learn: 0.1845932	test: 0.4576062	best: 0.4329959 (200)	total: 4.43s	remaining: 2.13s
675:	learn: 0.1845698	test: 0.4576281	best: 0.4329959 (200)	total: 4.44s	remaining: 2.13s
676:	learn: 0.1841689	test: 0.4574993	best: 0.4329959 (200)	total: 4.45s	remaining: 2.12s
677:	learn: 0.1837736	test: 0.4582684	best: 0.4329959 (200)	total: 4.46s	remaining: 2.12s
678:	learn: 0.1834100	test: 0.4582435	best: 0.4329959 (200)	total: 4.47s	remaining: 2.11s
679:	learn: 0.1833353	test: 0.4583824	best: 0.4329959 (200)	total: 4.47s	remaining: 2.1s
680:	learn: 0.1831506	test: 0.4589973	best: 0.4329959 (200)	total: 4.48s	remaining: 2.1s
681:	learn: 0.1828841	test: 0.4596078	best: 0.4329959 (200)	total: 4.48s	remaining: 2.09s
682:	learn: 0.1823638	test: 0.4596259	best: 0.4329959 (200)	total: 4.49s	remaining: 2.08s
683:	learn: 0

773:	learn: 0.1651828	test: 0.4662906	best: 0.4329959 (200)	total: 5.33s	remaining: 1.56s
774:	learn: 0.1649447	test: 0.4670887	best: 0.4329959 (200)	total: 5.34s	remaining: 1.55s
775:	learn: 0.1649126	test: 0.4671313	best: 0.4329959 (200)	total: 5.35s	remaining: 1.54s
776:	learn: 0.1647330	test: 0.4671245	best: 0.4329959 (200)	total: 5.36s	remaining: 1.54s
777:	learn: 0.1641826	test: 0.4665487	best: 0.4329959 (200)	total: 5.36s	remaining: 1.53s
778:	learn: 0.1640221	test: 0.4666968	best: 0.4329959 (200)	total: 5.37s	remaining: 1.52s
779:	learn: 0.1639610	test: 0.4666884	best: 0.4329959 (200)	total: 5.38s	remaining: 1.52s
780:	learn: 0.1639345	test: 0.4666411	best: 0.4329959 (200)	total: 5.38s	remaining: 1.51s
781:	learn: 0.1637145	test: 0.4664733	best: 0.4329959 (200)	total: 5.39s	remaining: 1.5s
782:	learn: 0.1636122	test: 0.4665241	best: 0.4329959 (200)	total: 5.39s	remaining: 1.5s
783:	learn: 0.1635082	test: 0.4666500	best: 0.4329959 (200)	total: 5.4s	remaining: 1.49s
784:	learn: 0

883:	learn: 0.1484172	test: 0.4796165	best: 0.4329959 (200)	total: 6.15s	remaining: 808ms
884:	learn: 0.1483751	test: 0.4796209	best: 0.4329959 (200)	total: 6.16s	remaining: 801ms
885:	learn: 0.1481575	test: 0.4789745	best: 0.4329959 (200)	total: 6.18s	remaining: 795ms
886:	learn: 0.1481397	test: 0.4790440	best: 0.4329959 (200)	total: 6.19s	remaining: 789ms
887:	learn: 0.1479454	test: 0.4792794	best: 0.4329959 (200)	total: 6.2s	remaining: 781ms
888:	learn: 0.1477090	test: 0.4797078	best: 0.4329959 (200)	total: 6.2s	remaining: 774ms
889:	learn: 0.1475559	test: 0.4796615	best: 0.4329959 (200)	total: 6.21s	remaining: 767ms
890:	learn: 0.1473669	test: 0.4797403	best: 0.4329959 (200)	total: 6.21s	remaining: 760ms
891:	learn: 0.1472550	test: 0.4795886	best: 0.4329959 (200)	total: 6.22s	remaining: 753ms
892:	learn: 0.1470853	test: 0.4794652	best: 0.4329959 (200)	total: 6.22s	remaining: 746ms
893:	learn: 0.1469820	test: 0.4794814	best: 0.4329959 (200)	total: 6.23s	remaining: 739ms
894:	learn: 

993:	learn: 0.1348787	test: 0.4858125	best: 0.4329959 (200)	total: 6.98s	remaining: 42.1ms
994:	learn: 0.1346008	test: 0.4859523	best: 0.4329959 (200)	total: 6.99s	remaining: 35.1ms
995:	learn: 0.1343902	test: 0.4860028	best: 0.4329959 (200)	total: 6.99s	remaining: 28.1ms
996:	learn: 0.1342712	test: 0.4859343	best: 0.4329959 (200)	total: 7s	remaining: 21.1ms
997:	learn: 0.1342389	test: 0.4859595	best: 0.4329959 (200)	total: 7.01s	remaining: 14ms
998:	learn: 0.1341691	test: 0.4859469	best: 0.4329959 (200)	total: 7.01s	remaining: 7.02ms
999:	learn: 0.1340561	test: 0.4858727	best: 0.4329959 (200)	total: 7.02s	remaining: 0us

bestTest = 0.4329958933
bestIteration = 200



In [10]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.82±0.01 on step 897


In [11]:
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))

Simple model validation accuracy: 0.8161


## 3 Using the Model

Let's apply the model to data in order to predict the labels. Also, we will have a look at CatBoost's feature importance.

In [12]:
# This will give the actual predictions
predictions = model.predict(X_test)
print(predictions[:10])


[0 0 0 0 1 0 1 0 1 0]


In [13]:
# And this the underlying probability values
predictions_probs = model.predict_proba(X_test)
print(predictions_probs[:10])

[[0.85473931 0.14526069]
 [0.76313031 0.23686969]
 [0.88972889 0.11027111]
 [0.87876173 0.12123827]
 [0.3611047  0.6388953 ]
 [0.90513381 0.09486619]
 [0.33434185 0.66565815]
 [0.78468564 0.21531436]
 [0.39429048 0.60570952]
 [0.94047549 0.05952451]]


### 3.1 Best Model
We can have a look at the difference between the simple CatBoost model and choosing the best model during the training iteration.

In [14]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

In [15]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))


Simple model validation accuracy: 0.7982

Best model validation accuracy: 0.8251


So there is a noticeable increase in accuracy on the validation set.

### Feature Importance

We will use CatBoost's feature ranking to see how the models differ. In a more complicated task with many more features, this could be very useful to interpret the model.

In [16]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
print("Feature importances for the baseline model")
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Feature importances for the baseline model
Sex: 21.59911877701903
Age: 19.398416335821832
Fare: 13.886969720468315
Pclass: 13.770872263093882
Embarked: 7.030284738986945
Ticket: 6.964771018892179
Parch: 6.355643060218354
SibSp: 6.010444835646993
Cabin: 4.983479249852257
PassengerId: 0.0
Name: 0.0


In [17]:
feature_importances = best_model.get_feature_importance(train_pool)
feature_names = X_train.columns
print("Feature importances for the best model")
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Feature importances for the best model
Sex: 42.17474818544473
Pclass: 16.294343982697637
Ticket: 9.797084805047552
Parch: 7.53610285333354
Age: 7.041597602831637
Fare: 4.9866064486621715
Cabin: 4.836829329849025
Embarked: 3.6878977676281557
SibSp: 3.6447890245054957
PassengerId: 0.0
Name: 0.0


PassengerId and Name have absolutely no effect on the outcome - which is good because there is no reason why they should. Compared to the aggregated baseline model, the best model attributes much more importance to the feature "Sex", while the importances of the features "Age" and "Fare" have been reduced noticeably.